Sync your Google Drive

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mount Google drive's root and drive folder

In [0]:
cd /content/gdrive/My Drive

In [0]:
ls -ltr

Select your own folder and Go into a folder

In [0]:
cd Colab Notebooks

Above codes are for Google drive sync.

If you are using your own Jupyter, just ignore it.




Start with Classification example

In [0]:
%matplotlib inline

## Load API
import matplotlib
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [0]:
## Set Autoencoder parameters
# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
batch_size = 128
epochs = 50

In [0]:
# the data, split between train and test sets
## Discard the output (since we're only interested in encoding/decoding the input images)
(x_train, _), (x_test, _) = mnist.load_data()

In [0]:
## Change 2-D image into 1-D Vector and  Normalized with range 0 - 1
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [0]:
## Build Autoencoder model
# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

## Show model Information
autoencoder.summary()

## Set train methodology
autoencoder.compile(loss='binary_crossentropy',
                    optimizer=RMSprop())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_2 (Dense)              (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [0]:
## Train model with train set.
autoencoder.fit(x_train, x_train,
                batch_size=batch_size,
                epochs=epochs,
                shuffle=True,
                validation_data=(x_test, x_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 6s 93us/step - loss: 0.2007 - val_loss: 0.1391
Epoch 2/50
60000/60000 [==============================] - 5s 80us/step - loss: 0.1265 - val_loss: 0.1151
Epoch 3/50
60000/60000 [==============================] - 5s 77us/step - loss: 0.1109 - val_loss: 0.1050
Epoch 4/50
60000/60000 [==============================] - 5s 77us/step - loss: 0.1038 - val_loss: 0.1007
Epoch 5/50
60000/60000 [==============================] - 5s 89us/step - loss: 0.1002 - val_loss: 0.0986
Epoch 6/50
60000/60000 [==============================] - 6s 97us/step - loss: 0.0983 - val_loss: 0.0964
Epoch 7/50
60000/60000 [==============================] - 5s 91us/step - loss: 0.0972 - val_loss: 0.0955
Epoch 8/50
60000/60000 [==============================] - 6s 101us/step - loss: 0.0965 - val_loss: 0.0953
Epoch 9/50
60000/60000 [==============================] - 6s 

## Evaluation - to visualize the reconstructed inputs and the encoded representation 

### create a separate encoder model

In [0]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

### create a separate decoder model

In [0]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [0]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

### Visualization

In [0]:
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()